In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [992 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [98.9 kB]
Get:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bion

In [18]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-11-05 00:07:31--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.45MB/s    in 0.2s    

2022-11-05 00:07:32 (5.45 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [97]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("PlaysETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [112]:
# Read in data from GCS Buckets
from pyspark import SparkFiles
url="https://storage.googleapis.com/big-data-bowl/plays.csv"
spark.sparkContext.addFile(url)
plays_data_df = spark.read.csv(SparkFiles.get("plays.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
plays_data_df.show()

+----------+------+--------------------+-------+----+---------+--------------+-------------+------------+--------------+---------+----------------+-------------------+----------+------------+--------------------+----------+--------------------+----------+---------+----------+---------+----------+----------------------+----------------+----------------+--------------+----------------+--------------------+--------------+----------------+--------------------+
|    gameId|playId|     playDescription|quarter|down|yardsToGo|possessionTeam|defensiveTeam|yardlineSide|yardlineNumber|gameClock|preSnapHomeScore|preSnapVisitorScore|passResult|penaltyYards|prePenaltyPlayResult|playResult|           foulName1|foulNFLId1|foulName2|foulNFLId2|foulName3|foulNFLId3|absoluteYardlineNumber|offenseFormation|      personnelO|defendersInBox|      personnelD|        dropBackType|pff_playAction|pff_passCoverage|pff_passCoverageType|
+----------+------+--------------------+-------+----+---------+--------------+

In [113]:
from pyspark.sql.types import IntegerType, BooleanType
plays_data_df = plays_data_df.withColumn("penaltyYards", plays_data_df["penaltyYards"].cast(IntegerType()))
plays_data_df = plays_data_df.withColumn("prePenaltyPlayResult", plays_data_df["prePenaltyPlayResult"].cast(IntegerType()))
plays_data_df = plays_data_df.withColumn("playResult", plays_data_df["playResult"].cast(IntegerType()))
plays_data_df = plays_data_df.withColumn("foulNFLId1", plays_data_df["foulNFLId1"].cast(IntegerType()))
plays_data_df = plays_data_df.withColumn("foulNFLId2", plays_data_df["foulNFLId2"].cast(IntegerType()))
plays_data_df = plays_data_df.withColumn("foulNFLId3", plays_data_df["foulNFLId3"].cast(IntegerType()))
plays_data_df = plays_data_df.withColumn("absoluteYardlineNumber", plays_data_df["absoluteYardlineNumber"].cast(IntegerType()))
plays_data_df = plays_data_df.withColumn("defendersInBox", plays_data_df["defendersInBox"].cast(IntegerType()))
plays_data_df = plays_data_df.withColumn("pff_playAction", plays_data_df["pff_playAction"].cast(IntegerType()))

from pyspark.sql.functions import *
plays_data_df = plays_data_df.withColumn('foulName1', regexp_replace('foulName1', 'NA', ''))
plays_data_df = plays_data_df.withColumn('foulName2', regexp_replace('foulName2', 'NA', ''))
plays_data_df = plays_data_df.withColumn('foulName3', regexp_replace('foulName3', 'NA', ''))
plays_data_df.show()

+----------+------+--------------------+-------+----+---------+--------------+-------------+------------+--------------+---------+----------------+-------------------+----------+------------+--------------------+----------+--------------------+----------+---------+----------+---------+----------+----------------------+----------------+----------------+--------------+----------------+--------------------+--------------+----------------+--------------------+
|    gameId|playId|     playDescription|quarter|down|yardsToGo|possessionTeam|defensiveTeam|yardlineSide|yardlineNumber|gameClock|preSnapHomeScore|preSnapVisitorScore|passResult|penaltyYards|prePenaltyPlayResult|playResult|           foulName1|foulNFLId1|foulName2|foulNFLId2|foulName3|foulNFLId3|absoluteYardlineNumber|offenseFormation|      personnelO|defendersInBox|      personnelD|        dropBackType|pff_playAction|pff_passCoverage|pff_passCoverageType|
+----------+------+--------------------+-------+----+---------+--------------+

Postgres Setup

In [34]:
# Store environment variable
from getpass import getpass
password = getpass('Enter database password')

# Configure settings for Cloud SQL
mode = "append"
jdbc_url="jdbc:postgresql://34.72.136.99:5432/big-data-bowl"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}


Enter database password··········


In [114]:
# Write DataFrame to plays table in Cloud SQL
plays_data_df.write.jdbc(url=jdbc_url, table='plays', mode=mode, properties=config)